# Configuração de ambiente

Importação das bibliotecas:
- Pandas: para manipulação de dados através do alias "pd"
- Google Colab: para manipulação de arquivos no Google Drive

E execução do comando "%matplotlib inline" para que os gráficos sejam exibidos no notebook logo abaixo da célula de código que os gerou.

In [ ]:
import pandas as pd
from google.colab import drive

%matplotlib inline